In [1]:
import json
import os

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import concurrent.futures


In [274]:
# df = pd.read_csv('data/players_15.csv')
df = pd.read_csv('data/players_20.csv')

In [275]:
player_urls = list(df['player_url'])

In [276]:
player_urls_new = []

In [277]:
for url in player_urls:
    new_url = '/'.join(url.split('/')[:-2]) + '/live'
    player_urls_new.append(new_url)
df['player_urls_new'] = player_urls_new

In [278]:
df['player_urls_new'][11]

'https://sofifa.com/player/201024/kalidou-koulibaly/live'

In [279]:
def get_teams_player_played_for(player_url, idx):
    page_html = requests.get(player_url).content
    page_html_soup = BeautifulSoup(page_html, 'html.parser')
    teams_table = page_html_soup.find_all(class_='team')
    teams = set()
    for item in teams_table[2:]:
        teams.add(item.text)
#     print(df['long_name'][idx]), print(teams)
    return df['long_name'][idx],  list(teams)
#     return teams

In [281]:
# test get_teams_player_played_for function
idx = 11
result= get_teams_player_played_for(df['player_urls_new'][idx], idx)
result

('Kalidou Koulibaly', ['Genk', 'Metz II', 'Napoli', 'Metz'])

In [215]:
#sequential 
# player_teams_played_for = []
# for idx, url in tqdm(enumerate(player_urls_new[:30])):
#     teams = get_teams_player_played_for(url, idx)
#     player_teams_played_for.append(teams)
# df['player_teams_played_for'] =  player_teams_played_for 
    

In [268]:
#parallel computation using threads
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = []
    for idx, url in tqdm(enumerate(player_urls_new[:800])):
        results.append(executor.submit(get_teams_player_played_for, url, idx))
    final_results = []
    for f in concurrent.futures.as_completed(results):
        final_results.append(f.result())

800it [00:00, 6323.42it/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [269]:
df_small = pd.DataFrame(columns=['player', 'teams'])

In [270]:
player_names = []
player_teams = []
for data in final_results:
    player_name, teams = data
    player_names.append(player_name)
    player_teams.append(teams)

df_small['player'] = player_names
df_small['teams']  = player_teams
df_small.to_pickle('data/data_small.pkl')
df_small.to_csv('data/data_small.csv')

In [271]:
idx = 11
df_small.iloc[idx]['player'], df_small.iloc[idx]['teams']

('Gareth Frank Bale',
 ['Tottenham Hotspur', 'Real Madrid', 'Tottenham', 'Southampton'])